# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [140]:
#create array of unique school names
unique_school_names = school_data['school_name'].unique()

#gives the length of unique school names to give us how many schools
school_count = len(unique_school_names)

#district student count
dist_student_count = school_data['size'].sum()

#student count from student file (to verify with district student count)
total_student_rec = student_data['school_name'].count()

#total budget
total_budget = school_data['budget'].sum()

#calculations for number and % passing reading
num_passing_reading = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = (num_passing_reading/total_student_rec)*100

#calculations for number and % passing math
num_passing_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
perc_pass_math = (num_passing_math/total_student_rec)*100


#Calculate the average math score
avg_math = student_data["math_score"].mean()


#Calculate the average reading score
avg_reading = student_data["reading_score"].mean()


#Calculate the overall pass rate
overall_pass = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]['student_name'].count()/total_student_rec


# district dataframe from dictionary

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading],
    "Average Math Score": [avg_math],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]

})

#format cells
district_summary.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:,.2f}", 
                       "Average Math Score": "{:,.2f}", 
                       "% Passing Math": "{:,.2f}", 
                       "% Passing Reading": "{:,.2f}", 
                       "Overall Passing Rate": "{:,.2%}"})

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$24,649,428.00",81.88,78.99,85.81,74.98,65.17%


## School Summary
* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [164]:
#group by school
grouped_school_data = school_data_complete.groupby(['school_name', "type"])

#total students by school
total_students = grouped_school_data["Student ID"].count()

#budget per school
total_budget = grouped_school_data["budget"].first()

#per student budget
per_student_budget =  (total_budget/total_students)

##avg scores by school
school_avgmath = grouped_school_data["math_score"].mean()

school_avgread = grouped_school_data["reading_score"].mean()

#passing scores by school
school_pass_math = (school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/total_students)*100 
school_pass_read = (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/total_students)*100
school_overallpass = (school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/total_students)*100 

#convert groupby object into a dataframe
grouped_school_data = pd.DataFrame({"Total Students":total_students,
                                    "Total School Budget": total_budget, 
                                    "Per Student Budget": per_student_budget,
                                    "Average Math Score": school_avgmath,
                                    "Average Reading Score": school_avgread,   
                                    "% Passing Math": school_pass_math,
                                    "% Passing Reading": school_pass_read,
                                    "Overall Passing Rate (%)": school_overallpass})

# Formatting dataframe
grouped_school_data['Total School Budget'] = grouped_school_data['Total School Budget'].map('${:,.2f}'.format)
grouped_school_data['Per Student Budget'] = grouped_school_data['Per Student Budget'].map('${:,.2f}'.format)
grouped_school_data['Average Math Score'] = grouped_school_data['Average Math Score'].map('{:,.2f}'.format)
grouped_school_data['Average Reading Score'] = grouped_school_data['Average Reading Score'].map('{:,.2f}'.format)
grouped_school_data['% Passing Math'] = grouped_school_data['% Passing Math'].map('{:,.2f}'.format)
grouped_school_data['% Passing Reading'] = grouped_school_data['% Passing Reading'].map('{:,.2f}'.format)
grouped_school_data['Overall Passing Rate (%)'] = grouped_school_data['Overall Passing Rate (%)'].map('{:,.2f}'.format)

grouped_school_data


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
school_name,type,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [165]:
#Sort and display the top five schools in overall passing rate
top_performing_schools = grouped_school_data.sort_values(by='Overall Passing Rate (%)',ascending=False)
top_performing_schools.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
school_name,type,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [167]:
#Sort and display the five worst-performing schools
worst_performing_schools = grouped_school_data.sort_values(by='Overall Passing Rate (%)')
worst_performing_schools.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
school_name,type,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [170]:
# Calculate the average math score for students of 9th grade at each school
school_avg_math_9th = school_data_complete[school_data_complete['grade']=='9th'].groupby('school_name')['math_score'].mean()
# Calculate the average math score for students of 10th grade at each school
school_avg_math_10th = school_data_complete[school_data_complete['grade']=='10th'].groupby('school_name')['math_score'].mean()
# Calculate the average math score for students of 11th grade at each school
school_avg_math_11th = school_data_complete[school_data_complete['grade']=='11th'].groupby('school_name')['math_score'].mean()
# Calculate the average math score for students of 12th grade at each school
school_avg_math_12th = school_data_complete[school_data_complete['grade']=='12th'].groupby('school_name')['math_score'].mean()

# Create a dataframe to hold the above results
grade_math_score={
    '9th':school_avg_math_9th,
    '10th':school_avg_math_10th,
    '11th':school_avg_math_11th,
    '12th':school_avg_math_12th,
    }

math_score_by_grade = pd.DataFrame(grade_math_score)
math_score_by_grade.index.name = None
math_score_by_grade


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [171]:
# Calculate the average reading score for students of 9th grade at each school
school_avg_reading_9th = school_data_complete[school_data_complete['grade']=='9th'].groupby('school_name')['reading_score'].mean()
# Calculate the average reading score for students of 10th grade at each school
school_avg_reading_10th = school_data_complete[school_data_complete['grade']=='10th'].groupby('school_name')['reading_score'].mean()
# Calculate the average reading score for students of 11th grade at each school
school_avg_reading_11th = school_data_complete[school_data_complete['grade']=='11th'].groupby('school_name')['reading_score'].mean()
# Calculate the average reading score for students of 12th grade at each school
school_avg_reading_12th = school_data_complete[school_data_complete['grade']=='12th'].groupby('school_name')['reading_score'].mean()

# Create a dataframe to hold the above results
grade_reading_score={
    '9th':school_avg_reading_9th,
    '10th':school_avg_reading_10th,
    '11th':school_avg_reading_11th,
    '12th':school_avg_reading_12th,
    }

reading_score_by_grade = pd.DataFrame(grade_reading_score)
reading_score_by_grade.index.name = None
reading_score_by_grade.head(20)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [187]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [197]:
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], spending_bins, labels = group_names)

#group by spending
by_spending = school_data_complete.groupby('spending_bins')

#calculations
avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()

# df build            
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_names)

#formating
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
<$585,83.4,84.0,93.7%,96.7%,90.6%
$585-615,83.5,83.8,94.1%,95.9%,90.1%
$615-645,78.1,81.4,71.4%,83.6%,60.3%
$645-675,77.0,81.0,66.2%,81.1%,53.5%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [227]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [259]:
school_data_complete['School Size'] = pd.cut(school_data_complete['size'], size_bins, labels = group_names)


by_size = school_data_complete.groupby('School Size')


avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('School Size')['Student ID'].count()/by_size['Student ID'].count() * 100
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('School Size')['Student ID'].count()/by_size['Student ID'].count() * 100

overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('School Size')['Student ID'].count()/by_size['Student ID'].count() * 100


scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate (%)": overall})

scores_by_size.index.name = "School Size"

scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1f}', 
                              '% Passing Reading':'{:.1f}', 
                              'Overall Passing Rate (%)': '{:.1f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
School Size,,,,,
Small (<1000),83.8,83.8,94.0,96.0,90.1
Medium (1000-2000),83.4,83.4,93.6,96.8,90.6
Large (2000-5000),77.5,77.5,68.7,82.1,56.6


## Scores by School Type

* Perform the same operations as above, based on school type.

In [260]:
# group by type of school
by_type = school_data_complete.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()*100
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()*100
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()*100

#Construct dataframe based on school type         
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate (%)": overall})

#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate (%)"]]

scores_by_type.index.name = "Type of School"

#formating
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1f}', 
                              '% Passing Reading':'{:.1f}', 
                              'Overall Passing Rate (%)': '{:.1f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
Type of School,,,,,
Charter,83.4,83.4,93.7,96.6,90.6
District,77.0,77.0,66.5,80.9,53.7
